# 02 - Data Analysis with Text-to-SQL: Leveraging Anthropic Claude 3 on Amazon Bedrock with Custom Prompts and SQLite

This notebook demonstrates a practical approach to enabling natural language querying of structured data using Large Language Models (LLMs) and SQLite. We leverage Anthropic's Claude to translate plain English questions into SQL queries, executed against a local SQLite database instance to mimic a SQL database without external dependencies. By combining LLM capabilities with SQL, we bridge the gap between non-technical users and data retrieval, enabling intuitive data exploration without the need for SQL proficiency. Techniques for prompt engineering and query optimization are also explored.

In contrast to the previous notebook, we delve deeper into how to customize and fine-tune the prompt to enhance the accuracy of the SQL queries.

In [1]:
!python --version

Python 3.10.13


In [2]:
%pip install --upgrade --quiet langchain langchain-community langchain-aws
%pip install -q sqlfluff

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Restart the kernel after installing dependencies

In [3]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import warnings
warnings.filterwarnings("ignore")

> This notebook was tested with a kernel with python 3.10.6

## 1. Explore data

The data is exported from the [country profile of Tunisia on Harvard Economic Atlas](https://atlas.cid.harvard.edu/explore?country=223&queryLevel=location&product=undefined&year=2021&productClass=HS&target=Product&partner=undefined&startYear=undefined)

In [5]:
import pandas as pd

In [6]:
ls data/

'What did Tunisia export in 2021_.csv'  'What did Tunisia import in 2021_.csv'


In [7]:
properties_data_df = pd.read_csv("data/What did Tunisia export in 2021_.csv")

In [8]:
properties_data_df.head()

,Name,Gross Export,Share,Code,Sector
0,Horses,109905,0.000539,0101,Agriculture
1,Fowl,284920,0.001396,0105,Agriculture
2,Other live animals,36664,0.000180,0106,Agriculture
3,Poultry,3507712,0.017193,0207,Agriculture
4,Other meat,1087,0.000005,0208,Agriculture


In [9]:
properties_data_df.shape

(976, 5)

In [10]:
properties_data_df["year"] = 2021

## 2. Important Design Choices and Decisions

Allowing large language models to prepare SQL queries to run against a database should be done with great caution and appropriate safeguards in place. We also need to recognize and work around the variety of database systems and SQL syntax variations when developing AI assistants that may access databases.

To achieve this, please consider the following recommendations when designing agents/assistants that can access SQL databases:

1. **Never allow the agent to perform write operations** to reduce the risk of SQL injection attacks.
2. The LLM agent should run with a user that has **read-only permission against a selection of tables**.
   - This reinforces and emphasizes point 1.
3. When possible, **use aggregated or materialized views** instead of querying the base tables directly. This will speed up the queries and reduce the load on the database.
4. **Limit the agent's access to only the required SQL tables** based on the expected user queries.

To control access to the database during the engagement, we will help you build a Lambda function that wraps the functionality of the LLM assistant. This Lambda function can be easily integrated with the remaining components of your system. By limiting the access permissions of the Lambda function to read-only from the SQL database, we can effectively limit the permissions of the LLM assistant.

## 3. Store data as SQLite db

In this demonstration, we use SQLite to mock the SQL database. You can validate your initial proof of concept using SQLite, then move to another production grade database for the pilot.

In [11]:
import sqlite3

# Create an empty SQLite db
conn = sqlite3.connect("/tmp/tunisia-exports.db")
c = conn.cursor()
# Write the pandas dataframe data into the SQLite db
properties_data_df.to_sql("tunisia_exports_table", conn, if_exists="replace", index=False)
conn.close()

## 3. Prepare an LLM 

In [12]:
import boto3

bedrock_client = boto3.client("bedrock", region_name="us-west-2")
bedrock_runtime_client = boto3.client("bedrock-runtime", region_name="us-west-2")

In [13]:
available_foundation_models = bedrock_client.list_foundation_models()

Uncomment the following to see the full list of models on Amazon Bedrock

In [14]:
# available_foundation_models

Below we keep only models from the Anthropic Claude family.

In [15]:
claude_models_on_bedrock = [
    m for m in available_foundation_models["modelSummaries"]
    if "Claude" in m["modelName"]
]

Below we extract the model ids for models in the Anthropic Claude family.

In [16]:
[m["modelId"] for m in claude_models_on_bedrock]

['anthropic.claude-instant-v1:2:100k',
 'anthropic.claude-instant-v1',
 'anthropic.claude-v2:0:18k',
 'anthropic.claude-v2:0:100k',
 'anthropic.claude-v2:1:18k',
 'anthropic.claude-v2:1:200k',
 'anthropic.claude-v2:1',
 'anthropic.claude-v2',
 'anthropic.claude-3-sonnet-20240229-v1:0:28k',
 'anthropic.claude-3-sonnet-20240229-v1:0:200k',
 'anthropic.claude-3-sonnet-20240229-v1:0',
 'anthropic.claude-3-haiku-20240307-v1:0:48k',
 'anthropic.claude-3-haiku-20240307-v1:0:200k',
 'anthropic.claude-3-haiku-20240307-v1:0',
 'anthropic.claude-3-opus-20240229-v1:0']

We pick the Claude haiku model for the first experiment.

In [17]:
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

In [18]:
from langchain_aws import ChatBedrock

In [19]:
model_kwargs = {
    "temperature": 0.0,
    "top_p": 0.99,
    "max_tokens": 1000,
}

llm = ChatBedrock(
    client=bedrock_runtime_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

In [20]:
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
Given an input question, write a syntactically correct {dialect} query that will be parsed out, validated for systax errors, and run automatically against a database.
The question is provided inside a <question> xml tag. Generate the SQL query inside a <sql_query> xml tag.

Please only use the following tables inside <table_info>:

<table_info>
{table_info}.
</table_info>

1. You must limit the query to {top_k}
2. If you need to create an alias use snake case format, e.g new_alias.
"""

messages = [
    ("system", system_prompt),
    ("user", "<question>{input}</question>")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [21]:
prompt

ChatPromptTemplate(input_variables=['dialect', 'input', 'table_info', 'top_k'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'table_info', 'top_k'], template='\nGiven an input question, write a syntactically correct {dialect} query that will be parsed out, validated for systax errors, and run automatically against a database.\nThe question is provided inside a <question> xml tag. Generate the SQL query inside a <sql_query> xml tag.\n\nPlease only use the following tables inside <table_info>:\n\n<table_info>\n{table_info}.\n</table_info>\n\n1. You must limit the query to {top_k}\n2. If you need to create an alias use snake case format, e.g new_alias.\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='<question>{input}</question>'))])

In [22]:
from langchain import SQLDatabase

In [23]:
# load db
tunisia_exports_db = SQLDatabase.from_uri("sqlite:////tmp/tunisia-exports.db")

## Ask the LLM to Generate SQL Queries then Handle the Execution Separately

### Generate the SQL Query from Natural Language

In [24]:
from langchain.chains import create_sql_query_chain

In [25]:
text_to_sql_chain = create_sql_query_chain(llm=llm, db=tunisia_exports_db, prompt=prompt)

In [26]:
user_question = "Which sector has the highest total gross expert in Tunisia?"

In [27]:
%%time

sql_query = text_to_sql_chain.invoke({"question": user_question})
sql_query

CPU times: user 56.8 ms, sys: 7.52 ms, total: 64.3 ms
Wall time: 1.31 s


'<sql_query>\nSELECT \n    t.Sector,\n    SUM(t."Gross Export") AS total_gross_export\nFROM tunisia_exports_table t\nGROUP BY t.Sector\nORDER BY total_gross_export DESC\nLIMIT 1;\n</sql_query>'

In [28]:
from bs4 import BeautifulSoup


def parse_query(response):
    soup = BeautifulSoup(response, 'html.parser')
    sql_query_tag = soup.find('sql_query')
    sql_query_text = sql_query_tag.get_text()
    return sql_query_text.strip()

In [29]:
parse_query(sql_query)

'SELECT \n    t.Sector,\n    SUM(t."Gross Export") AS total_gross_export\nFROM tunisia_exports_table t\nGROUP BY t.Sector\nORDER BY total_gross_export DESC\nLIMIT 1;'

### Apply a SQL Linter to Automatically Fix the Query if There is a Need

After receiving the SQL query, you can validate it, execute it, and potentially call an LLM with the original question and the query answer to formulate a natural language answer. This gives you full control over the query and question answering lifecycle.

You can, for example, run a SQL linter such as [sqlfluff](https://github.com/sqlfluff/sqlfluff) on the SQL query.

In [30]:
%%time
import sqlfluff

fixed_query = sqlfluff.fix(
    sql=parse_query(sql_query),
    dialect='postgres'
) 

print(fixed_query)

SELECT
    t.sector,
    SUM(t."Gross Export") AS total_gross_export
FROM tunisia_exports_table AS t
GROUP BY t.sector
ORDER BY total_gross_export DESC
LIMIT 1;

CPU times: user 627 ms, sys: 62.3 ms, total: 690 ms
Wall time: 1.2 s


### Execute the Query Against the Database Manually

In [31]:
%%time
conn = sqlite3.connect("/tmp/tunisia-exports.db")
c = conn.cursor()
c.execute(fixed_query)
result = c.fetchall()
result[0]
conn.close()

CPU times: user 186 µs, sys: 2.44 ms, total: 2.63 ms
Wall time: 2.91 ms


In [32]:
result

[('Electronics', 4533943713)]

In [33]:
conn = sqlite3.connect("/tmp/tunisia-exports.db")


def execute_sql_query(conn, query):
    c = conn.cursor()
    c.execute(query)
    result = c.fetchall()
    result[0]
    return result

### Formulate a Natural Language Answer from the Query and Result

In [34]:
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


system_prompt = """
You are an AI assistant helping non-technical business users understand data insights clearly and professionally. Your task is to take a business question and SQL query results, and provide a concise answer addressing the question directly, without delving into technical details.

Follow these guidelines:

1. Understand the context of the question and data.
2. Craft a clear, confident answer in a professional tone.
3. Focus on key insights and takeaways relevant to the question.
4. Use simple, non-technical language.
5. Highlight the most important points for quick understanding.

Your goal is to communicate actionable insights that empower informed business decisions based on the data, without overwhelming with technical jargon.

Provide a concise and professional answer based on the question and SQL results."""

messages = [
    ("system", system_prompt),
    ("user", "<question>{question}</question><sql_result>{sql_result}</sql_result>")
]

prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | llm | StrOutputParser()

In [35]:
%%time
response = chain.invoke(
    {
        "question": user_question,
        "sql_result": json.dumps(result)
    }
)

CPU times: user 12.9 ms, sys: 0 ns, total: 12.9 ms
Wall time: 2.22 s


In [36]:
print(response)

Based on the SQL query results, the sector with the highest total gross export in Tunisia is Electronics, with a total gross export value of 4,533,943,713.

The key insight here is that the Electronics sector is the top exporting industry in Tunisia, generating significantly more export revenue than other sectors. This suggests the Electronics industry is a major driver of Tunisia's export economy and could be an area of focus for further growth and investment.


## Evaluate on multiple questions

In [37]:
questions = [
    "Which sector has the highest total gross export in Tunisia?",
    "What are the top 3 sectors with the highest gross export in Tunisia?"
]

In [38]:
%%time

qa = []
for question in questions:
    sql_query = text_to_sql_chain.invoke({"question": question})
    sql_query = parse_query(sql_query)
    result = execute_sql_query(conn, sql_query)
    response = chain.invoke(
        {
            "question": question,
            "sql_result": json.dumps(result)
        }
    )
    qa.append(dict(question=question, answer=response.strip(), sql_query=sql_query))

CPU times: user 85.4 ms, sys: 2.25 ms, total: 87.6 ms
Wall time: 6.45 s


In [39]:
qa

[{'question': 'Which sector has the highest total gross export in Tunisia?',
  'answer': "Based on the SQL query results, the sector with the highest total gross export in Tunisia is Electronics, with a total export value of 4,533,943,713.\n\nThe key insight here is that the Electronics sector is the top exporting industry in Tunisia, significantly outpacing other sectors. This suggests that the Electronics industry is a major driver of Tunisia's export economy and likely plays a critical role in the country's overall economic performance.",
  'sql_query': 'SELECT \n    t.Sector,\n    SUM(t."Gross Export") AS total_gross_export\nFROM tunisia_exports_table t\nGROUP BY t.Sector\nORDER BY total_gross_export DESC\nLIMIT 1;'},
 {'question': 'What are the top 3 sectors with the highest gross export in Tunisia?',
  'answer': "Based on the SQL query results, the top 3 sectors with the highest gross export in Tunisia are:\n\n1. Electronics - $4,533,943,713\n2. Textiles - $4,099,307,399 \n3. Ser

In [40]:
index = 0
print(qa[index]["question"] + "\n\n", qa[index]["answer"].strip() + "\n\n", qa[index]["sql_query"].strip())

Which sector has the highest total gross export in Tunisia?

 Based on the SQL query results, the sector with the highest total gross export in Tunisia is Electronics, with a total export value of 4,533,943,713.

The key insight here is that the Electronics sector is the top exporting industry in Tunisia, significantly outpacing other sectors. This suggests that the Electronics industry is a major driver of Tunisia's export economy and likely plays a critical role in the country's overall economic performance.

 SELECT 
    t.Sector,
    SUM(t."Gross Export") AS total_gross_export
FROM tunisia_exports_table t
GROUP BY t.Sector
ORDER BY total_gross_export DESC
LIMIT 1;


In [41]:
index = 1
print(qa[index]["question"] + "\n\n", qa[index]["answer"].strip() + "\n\n", qa[index]["sql_query"].strip())

What are the top 3 sectors with the highest gross export in Tunisia?

 Based on the SQL query results, the top 3 sectors with the highest gross export in Tunisia are:

1. Electronics - $4,533,943,713
2. Textiles - $4,099,307,399 
3. Services - $2,933,225,232

The electronics sector has the highest gross export value, followed by textiles and then services. These appear to be the key export-oriented industries driving Tunisia's economy.

 SELECT 
    t.Sector,
    SUM(t."Gross Export") AS total_gross_export
FROM tunisia_exports_table t
GROUP BY t.Sector
ORDER BY total_gross_export DESC
LIMIT 3;


## Improve by Thinking Aloud, Allowing Refusals, and Static Analysis

For complex queries, there may be a need to iteratively refine the prompt to arrive at a correct SQL query. This iterative process can be facilitated by incorporating feedback and analysis from various sources. Here are a few approaches we can consider:

1. **LLM-Driven Iterative Query Generation**: Ask the LLM to generate an initial SQL query based on the user input. Evaluate this query for syntax correctness and semantic validity with respect to the user's intent. Provide this feedback to the LLM and prompt it to generate an updated version of the query. This cycle can be repeated until a satisfactory query is generated.

2. **Static Analysis-Assisted Query Refinement**: After receiving the LLM-generated SQL query, perform static analysis to identify potential errors or issues. This could include checking for syntax errors, type mismatches, missing table or column references, etc. Provide the LLM with the original query and the error messages or analysis results, and ask it to refine the query accordingly.

3. **LLM Agent for SQL Query Generation**: Leverage an LLM agent specifically designed for SQL query generation. Such an agent can handle the iterative process of generating, executing, and refining SQL queries automatically. Upon encountering errors or issues during query execution, the agent can autonomously prompt the LLM for an improved query, leveraging the execution feedback and any other relevant context.

In addition to these approaches, it may be beneficial to incorporate domain knowledge, data schema information, and examples of correct queries to guide the LLM's understanding of the problem space. Providing clear and structured prompts, along with iterative refinement, can help the LLM generate more accurate and reliable SQL queries, especially for complex scenarios.

### Allow Refusal

Below we tune the previous prompt to implement the first idea:

In [42]:
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
Given an input question, write a syntactically correct SQL query in {dialect} that will be parsed out, validated for syntax errors, and run automatically against a database.
The question is provided inside a <question> xml tag. Generate the SQL query inside a <sql_query> xml tag.

Please follow these guidelines:

1. You must limit the query to {top_k} results.
2. If you need to create an alias, use snake case format, e.g., new_alias.
4. Use the following tables inside <table_info> xml tag.
5. Handle edge cases such as ambiguous or incomplete questions by putting .
6. Optimize queries for performance by using appropriate indexes, joins, or other techniques when applicable.
7. Subqueries, window functions, and other advanced SQL constructs are allowed.

<table_info> {table_info} </table_info>

<ambiguous>true or false</ambiguous>
<ambiguous_reason>explain why you are not able to generate the query when relevant</ambiguous_reason>.
"""

messages = [
    ("system", system_prompt),
    ("user", "<question>{input}</question>")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [43]:
prompt

ChatPromptTemplate(input_variables=['dialect', 'input', 'table_info', 'top_k'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['dialect', 'table_info', 'top_k'], template='\nGiven an input question, write a syntactically correct SQL query in {dialect} that will be parsed out, validated for syntax errors, and run automatically against a database.\nThe question is provided inside a <question> xml tag. Generate the SQL query inside a <sql_query> xml tag.\n\nPlease follow these guidelines:\n\n1. You must limit the query to {top_k} results.\n2. If you need to create an alias, use snake case format, e.g., new_alias.\n4. Use the following tables inside <table_info> xml tag.\n5. Handle edge cases such as ambiguous or incomplete questions by putting .\n6. Optimize queries for performance by using appropriate indexes, joins, or other techniques when applicable.\n7. Subqueries, window functions, and other advanced SQL constructs are allowed.\n\n<table_info> {table_in

In [44]:
text_to_sql_chain = create_sql_query_chain(llm=llm, db=tunisia_exports_db, prompt=prompt)

In [45]:
user_question = "Which sector has the highest total gross expert in Tunisia?"

In [46]:
%%time

sql_query = text_to_sql_chain.invoke({"question": user_question})
sql_query

CPU times: user 32.6 ms, sys: 136 µs, total: 32.7 ms
Wall time: 1.16 s


'<sql_query>\nSELECT \n  Sector,\n  SUM("Gross Export") AS total_gross_export\nFROM tunisia_exports_table\nGROUP BY Sector\nORDER BY total_gross_export DESC\nLIMIT 1;\n</sql_query>'

In [47]:
user_question = "What is the number of car exports in Tunisia?"

In [48]:
%%time

sql_query = text_to_sql_chain.invoke({"question": user_question})
sql_query

CPU times: user 36.1 ms, sys: 2 ms, total: 38.1 ms
Wall time: 2.27 s


'The given question is ambiguous as it does not specify the type of car exports. The `tunisia_exports_table` does not contain any information about car exports. Without more context about the specific type of car exports, it is not possible to generate a meaningful SQL query to answer this question.\n\n<ambiguous>true</ambiguous>\n<ambiguous_reason>The question does not provide enough information to determine the specific type of car exports to query from the given table.</ambiguous_reason>'

## Thinking Aloud

Below we ask the LLM to think aloud, by evaluating it's first attempt at generating the SQL query before the generating the final query.

In [49]:
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
Given an input question, write a syntactically correct SQL query in {dialect} that will be parsed out, validated for syntax errors, and run automatically against a database.
The question is provided inside a <question> xml tag. Generate the SQL query inside a <sql_query> xml tag.

Please follow these guidelines:

1. You must limit the query to {top_k} results.
2. If you need to create an alias, use snake case format, e.g., new_alias.
4. Use the following tables inside <table_info> xml tag.
5. Handle edge cases such as ambiguous or incomplete questions by putting .
6. Optimize queries for performance by using appropriate indexes, joins, or other techniques when applicable.
7. Subqueries, window functions, and other advanced SQL constructs are allowed.

<table_info> {table_info} </table_info>

<ambiguous>true or false</ambiguous>
<ambiguous_reason>explain why you are not able to generate the query when relevant</ambiguous_reason>.

When you have enough information to answer, follow the format below:

1. <first_sql_query>put your first attempt here</first_sql_query>
2. <evaluation>evaluate if the first sql query meets all requirements and is syntactically correct</evaluation>
3. <sql_query>put final sql query here</sql_query>
"""

messages = [
    ("system", system_prompt),
    ("user", "<question>{input}</question>")
]

prompt = ChatPromptTemplate.from_messages(messages)

In [50]:
text_to_sql_chain = create_sql_query_chain(llm=llm, db=tunisia_exports_db, prompt=prompt)

In [51]:
user_question = "Which sector has the highest total gross expert in Tunisia?"

In [52]:
%%time

sql_query = text_to_sql_chain.invoke({"question": user_question})
sql_query

CPU times: user 31.8 ms, sys: 0 ns, total: 31.8 ms
Wall time: 2.56 s


'<first_sql_query>\nSELECT Sector, SUM("Gross Export") AS total_gross_export\nFROM tunisia_exports_table\nGROUP BY Sector\nORDER BY total_gross_export DESC\nLIMIT 1;\n</first_sql_query>\n\n<evaluation>\nThe first SQL query meets all the requirements and is syntactically correct. It selects the Sector and calculates the total Gross Export for each sector, orders the results by the total_gross_export in descending order, and limits the output to 1 row, which will be the sector with the highest total gross export.\n</evaluation>\n\n<sql_query>\nSELECT Sector, SUM("Gross Export") AS total_gross_export\nFROM tunisia_exports_table\nGROUP BY Sector\nORDER BY total_gross_export DESC\nLIMIT 1;\n</sql_query>'

# Conclusion

In this notebook, we demonstrated how to set up a dataframe of fake data, store it in a SQLite database to mimic a SQL database, and use an LLM chatbot to generate SQL queries based on natural language questions. We also explored techniques for executing the generated queries, validating them using a SQL linter, and formulating natural language answers based on the query results. The notebook provides a solid foundation for building AI assistants capable of querying databases while adhering to best practices for security and performance.

---

### Environment and Dependency Information

In [53]:
from utils.helper import package_imports
dict(package_imports(globals()))

{'builtins': None,
 'IPython.core.interactiveshell': '8.20.0',
 'IPython.core.autocall': '8.20.0',
 'io': None,
 'IPython.core.display': '8.20.0',
 'warnings': None,
 'pandas': '2.1.4',
 'pandas.core.frame': '2.1.4',
 'sqlite3': None,
 'boto3': '1.34.93',
 'botocore.client': '1.34.93',
 'langchain_aws.chat_models.bedrock': None,
 'json': '2.0.9',
 'langchain_core.output_parsers.string': '0.1.46',
 'langchain_core.prompts.chat': '0.1.46',
 'langchain_community.utilities.sql_database': '0.0.34',
 'langchain.chains.sql_database.query': '0.1.16',
 'langchain_core.runnables.base': '0.1.46',
 'bs4': '4.12.3',
 '__main__': None,
 'sqlfluff': '3.0.5',
 'utils.helper': None}

In [54]:
!uname -a

Linux default 4.14.336-257.562.amzn2.x86_64 #1 SMP Sat Feb 24 09:50:35 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux


In [55]:
!python --version

Python 3.10.13
